# 找出Susquehanna区域中的GAGES-II和CAMELS流域

In [1]:
import geopandas as gpd
import pandas as pd
import os

fp = os.path.join("data", "3557basins_ID_NSE_DOR.csv")
# Read file using gpd.read_file()
data = pd.read_csv(fp, dtype={"GAUGE ID": str})
data.head()

,GAUGE ID,NSE,DOR
0,01011000,0.89,0.06
1,01013500,0.90,0.00
2,01015800,0.89,0.02
3,01017000,0.90,0.04
4,01019000,0.41,0.97


In [2]:
data.shape

(3557, 3)

下载流域边界shapefile: https://water.usgs.gov/GIS/dsdl/boundaries_shapefiles_by_aggeco.zip

解压后文件夹放入 data 文件夹。

下面先试试一个shpfile中筛选：

In [3]:
shpfile_dir = os.path.join("data", "boundaries-shapefiles-by-aggeco")
shpfile1 = os.path.join(shpfile_dir, "bas_nonref_CntlPlains.shp")
shp1 = gpd.read_file(shpfile1)
shp1.head()

,AREA,PERIMETER,GAGE_ID,geometry
0,1.209580e+08,81060.0,03144816,"POLYGON ((1123005.000 1972005.000, 1123035.000..."
1,3.458990e+08,158340.0,03145000,"POLYGON ((1123005.000 1972005.000, 1123035.000..."
2,2.573460e+07,38760.0,03156000,"POLYGON ((1130325.000 1933275.000, 1130325.000..."
3,2.288740e+08,115800.0,03157000,"POLYGON ((1123755.000 1919955.000, 1123725.000..."
4,1.194710e+09,290820.0,03157500,"POLYGON ((1123755.000 1919955.000, 1123725.000..."


In [4]:
shp1_chosen = shp1.loc[shp1["GAGE_ID"].isin(data["GAUGE ID"].values)]
shp1_chosen

,AREA,PERIMETER,GAGE_ID,geometry
1,3.458990e+08,158340.0,03145000,"POLYGON ((1123005.000 1972005.000, 1123035.000..."
3,2.288740e+08,115800.0,03157000,"POLYGON ((1123755.000 1919955.000, 1123725.000..."
4,1.194710e+09,290820.0,03157500,"POLYGON ((1123755.000 1919955.000, 1123725.000..."
5,1.469500e+09,497040.0,03219500,"POLYGON ((1012575.000 2026005.000, 1012575.000..."
6,4.639170e+08,214200.0,03220000,"POLYGON ((1046745.000 1997475.000, 1046775.000..."
...,...,...,...,...
966,2.998490e+10,2148660.0,07191500,"POLYGON ((142995.000 1611525.000, 142995.000 1..."
970,1.088450e+09,335280.0,03324300,"POLYGON ((886590.000 2014470.000, 886590.000 2..."
974,9.246230e+08,324000.0,05061000,"POLYGON ((-13395.000 2661705.000, -13395.000 2..."
975,1.232660e+09,294960.0,05061500,"POLYGON ((-12165.000 2643525.000, -12165.000 2..."


下面看看两个：

In [5]:
shpfile2 = os.path.join(shpfile_dir, "bas_nonref_EastHghlnds.shp")
shp2 = gpd.read_file(shpfile2)
shp2_chosen = shp2.loc[shp2["GAGE_ID"].isin(data["GAUGE ID"].values)]
chosen12 = [shp1_chosen, shp2_chosen]
rdf = gpd.GeoDataFrame( pd.concat( chosen12, ignore_index=True) )
rdf

,AREA,PERIMETER,GAGE_ID,geometry
0,3.458990e+08,158340.0,03145000,"POLYGON ((1123005.000 1972005.000, 1123035.000..."
1,2.288740e+08,115800.0,03157000,"POLYGON ((1123755.000 1919955.000, 1123725.000..."
2,1.194710e+09,290820.0,03157500,"POLYGON ((1123755.000 1919955.000, 1123725.000..."
3,1.469500e+09,497040.0,03219500,"POLYGON ((1012575.000 2026005.000, 1012575.000..."
4,4.639170e+08,214200.0,03220000,"POLYGON ((1046745.000 1997475.000, 1046775.000..."
...,...,...,...,...
782,3.869600e+08,179640.0,03052000,"POLYGON ((1356705.000 1873515.000, 1356675.000..."
783,3.732210e+07,49260.0,03052500,"POLYGON ((1356945.000 1883115.000, 1356945.000..."
784,2.907300e+08,136920.0,03151400,"POLYGON ((1347915.000 1861065.000, 1347915.000..."
785,1.369110e+09,270600.0,03189100,"POLYGON ((1355415.000 1824675.000, 1355415.000..."


接下来就可以尝试多个了：

In [6]:
shpfiles = ["bas_nonref_CntlPlains.shp", "bas_nonref_EastHghlnds.shp", "bas_nonref_MxWdShld.shp", "bas_nonref_NorthEast.shp", "bas_nonref_SECstPlain.shp", 
            "bas_nonref_SEPlains.shp", "bas_nonref_WestMnts.shp", "bas_nonref_WestPlains.shp", "bas_nonref_WestXeric.shp", "bas_ref_all.shp"]
chosen_lst = []
for shpfile_name in shpfiles:
    shpfile = os.path.join(shpfile_dir,shpfile_name)
    shp = gpd.read_file(shpfile)
    shp_chosen = shp.loc[shp["GAGE_ID"].isin(data["GAUGE ID"].values)]
    chosen_lst.append(shp_chosen)
merge_chosen = gpd.GeoDataFrame( pd.concat( chosen_lst, ignore_index=True) )
merge_chosen

,AREA,PERIMETER,GAGE_ID,geometry
0,3.458990e+08,158340.0,03145000,"POLYGON ((1123005.000 1972005.000, 1123035.000..."
1,2.288740e+08,115800.0,03157000,"POLYGON ((1123755.000 1919955.000, 1123725.000..."
2,1.194710e+09,290820.0,03157500,"POLYGON ((1123755.000 1919955.000, 1123725.000..."
3,1.469500e+09,497040.0,03219500,"POLYGON ((1012575.000 2026005.000, 1012575.000..."
4,4.639170e+08,214200.0,03220000,"POLYGON ((1046745.000 1997475.000, 1046775.000..."
...,...,...,...,...
3552,7.939520e+08,218220.0,06280300,"POLYGON ((-1104450.000 2406630.000, -1104480.0..."
3553,1.629860e+08,92400.0,06632400,"POLYGON ((-848325.000 2106675.000, -848355.000..."
3554,3.983090e+08,166380.0,09210500,"POLYGON ((-1190715.000 2245455.000, -1190685.0..."
3555,7.164230e+08,156740.0,10308200,"POLYGON ((-2037620.625 1999732.625, -2037620.2..."


In [7]:
shp2.crs

<Projected CRS: PROJCS["NAD_1983_Albers",GEOGCS["NAD83",DATUM["Nor ...>
Name: NAD_1983_Albers
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

最后按id排序，并将结果输出到一个新的shpfile中。

In [8]:
sorted_merge_chosen = merge_chosen.sort_values(by=['GAGE_ID'], ignore_index=True)
sorted_merge_chosen

,AREA,PERIMETER,GAGE_ID,geometry
0,3.186830e+09,497160.0,01011000,"POLYGON ((2045745.000 2882355.000, 2045775.000..."
1,2.252690e+09,516060.0,01013500,"POLYGON ((2050770.000 2982930.000, 2050770.000..."
2,2.313750e+09,504060.0,01015800,"POLYGON ((2042445.000 2895585.000, 2042445.000..."
3,4.278900e+09,667800.0,01017000,"POLYGON ((2042445.000 2895585.000, 2042445.000..."
4,6.206210e+08,199620.0,01019000,"POLYGON ((2164515.000 2787285.000, 2164515.000..."
...,...,...,...,...
3552,1.252840e+09,244320.0,14366000,"POLYGON ((-2190585.000 2458935.000, -2190585.0..."
3553,1.809660e+09,287640.0,14369500,"POLYGON ((-2211165.000 2471835.000, -2211165.0..."
3554,1.019650e+10,872760.0,14372300,"POLYGON ((-2105205.000 2527365.000, -2105205.0..."
3555,9.853690e+08,236340.0,14377100,"POLYGON ((-2237505.000 2463945.000, -2237505.0..."


In [9]:
# Set the GeoDataFrame's coordinate system
sorted_merge_chosen.crs = shp2.crs

In [10]:
if not os.path.isdir("results"):
    os.mkdir("results")

In [11]:
sorted_merge_chosen.to_file(os.path.join("results", "3557basins.shp"))

QGIS看了一下3557basins和Susquehanna的相交情况，就选择几个有相交的流域就行，流域90%以上在Susquehanna中的都算Susquehanna里的。

In [12]:
shpfile_susquehanna = os.path.join("data", "HUC8_Susquehanna.shp")
shp_susquehanna = gpd.read_file(shpfile_susquehanna)
shp_susquehanna.head()

,HUC8,LoadDate,SourceFeat,AreaSqKm,GNIS_ID,TNMID,SourceOrig,SourceData,Name,Shape_Leng,States,MetaSource,OBJECTID,AreaAcres,Shape_Area,geometry
0,02050102,1339398000000,None,4162.80,0,{578672FB-862F-44D7-8910-C8A817E8E9FF},None,None,Chenango,4.955803,NY,None,21,1028650.56,0.456467,"POLYGON ((-76.27439 42.61470, -76.27425 42.614..."
1,02050106,1339398000000,None,5191.43,0,{5A2592DB-5A00-4ED8-8164-6484112CAF25},None,None,Upper Susquehanna-Tunkhannock,5.272169,PA,None,26,1282830.23,0.561352,"POLYGON ((-76.92545 41.63779, -76.92540 41.637..."
2,02050107,1339398000000,None,4572.75,0,{E105497D-9D5D-4ABD-841F-7FCFC7F645C2},None,None,Upper Susquehanna-Lackawanna,6.404887,PA,None,27,1129951.26,0.490690,"POLYGON ((-76.81377 40.94261, -76.81245 40.942..."
3,02050201,1339398000000,None,4136.01,0,{C1413C91-5A66-4655-9B2F-0A29F1C25DC3},None,None,Upper West Branch Susquehanna,4.276738,PA,None,28,1022029.15,0.442068,"POLYGON ((-78.91438 40.71609, -78.91424 40.716..."
4,02050202,1339398000000,None,2677.96,0,{8E629A6D-EC8F-49F7-804E-080C63A6F589},None,None,Sinnemahoning,3.646456,PA,None,29,661737.48,0.288513,"POLYGON ((-78.69231 41.20441, -78.69184 41.204..."


In [13]:
shp_susquehanna['new_column'] = 0
susquehanna_union = shp_susquehanna.dissolve(by='new_column')
susquehanna_union

,geometry,HUC8,LoadDate,SourceFeat,AreaSqKm,GNIS_ID,TNMID,SourceOrig,SourceData,Name,Shape_Leng,States,MetaSource,OBJECTID,AreaAcres,Shape_Area
new_column,,,,,,,,,,,,,,,,
0,"POLYGON ((-78.88628 39.91550, -78.88616 39.916...",02050102,1339398000000,None,4162.8,0,{578672FB-862F-44D7-8910-C8A817E8E9FF},None,None,Chenango,4.955803,NY,None,21,1028650.56,0.456467


In [14]:
susquehanna_union.to_file(os.path.join("results", "susquehanna_union.shp"))

In [15]:
print(susquehanna_union.crs)

GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]


In [16]:
print(sorted_merge_chosen.crs)

PROJCS["NAD_1983_Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [17]:
susquehanna_union = susquehanna_union.to_crs(sorted_merge_chosen.crs)

In [18]:
susquehanna_union.crs == sorted_merge_chosen.crs

True

In [19]:
basin3557_polygon_lst= [sorted_merge_chosen.geometry[i] for i in range(sorted_merge_chosen.shape[0])]

找到相交的流域

In [20]:
intersects_indices = [i for i in range(len(basin3557_polygon_lst)) if basin3557_polygon_lst[i].intersects(susquehanna_union.geometry[0])]

In [21]:
intersects_indices

[174,
 178,
 179,
 180,
 181,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 260,
 261,
 262,
 267,
 275,
 276,
 279,
 280,
 284,
 290,
 293,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 319,
 329,
 330,
 331,
 334,
 335,
 336,
 337,
 345,
 346,
 347,
 348,
 349,
 350,
 351,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 385,
 386,
 387,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 401,
 402,
 403,
 404,
 405,
 406,
 407,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 424,
 425,
 426,
 427,
 428,
 452,
 453,
 460,
 461,
 462,
 463,
 485,
 486,
 487,
 488,
 493,
 494,
 978,
 979,
 981,
 990,
 993,
 994,
 995,
 996,
 997,
 998,
 1000,
 1001,
 1002,
 1003,
 1005,
 1007,
 1029,
 1031,
 1032,
 1033,
 1475,
 1476,
 1477,
 1478,
 1479,
 1480,
 1481,
 1485

In [22]:
intersect_gdf = sorted_merge_chosen.iloc[intersects_indices]
intersect_gdf

,AREA,PERIMETER,GAGE_ID,geometry
174,3.450500e+09,666448.0,01347000,"POLYGON ((1703250.625 2478854.750, 1703229.875..."
178,4.375980e+07,45840.0,01350140,"POLYGON ((1734765.000 2359635.000, 1734765.000..."
179,1.153140e+09,247980.0,01350355,"POLYGON ((1744875.000 2369265.000, 1744875.000..."
180,2.320990e+09,470220.0,01351500,"POLYGON ((1731795.000 2372055.000, 1731765.000..."
181,9.113440e+09,1206350.0,01357500,"POLYGON ((1703250.625 2478854.750, 1703229.875..."
...,...,...,...,...
1493,3.316840e+08,160020.0,04240010,"POLYGON ((1600245.000 2395515.000, 1600215.000..."
1495,1.169270e+08,92400.0,04240180,"POLYGON ((1584405.000 2377335.000, 1584375.000..."
1496,2.894710e+08,203940.0,04240300,"POLYGON ((1587075.000 2382825.000, 1587075.000..."
1497,3.004510e+08,167520.0,04243500,"POLYGON ((1647285.000 2408295.000, 1647315.000..."


In [23]:
intersect_gdf["GAGE_ID"].values

array(['01347000', '01350140', '01350355', '01351500', '01357500',
       '01423000', '01425000', '01426500', '01427510', '01428500',
       '01428750', '01429000', '01430000', '01431500', '01434000',
       '01438500', '01446500', '01447500', '01447800', '01449000',
       '01451000', '01453000', '01463500', '01468500', '01469500',
       '01470500', '01470779', '01470960', '01471000', '01471510',
       '01472000', '01472157', '01474500', '01480300', '01480500',
       '01480617', '01480700', '01480870', '01481000', '01481500',
       '01495000', '01500000', '01503000', '01509000', '01510000',
       '01512500', '01516350', '01516500', '01518000', '01518700',
       '01518862', '01520000', '01521500', '01523500', '01524500',
       '01526500', '01529500', '01529950', '01531000', '01531500',
       '01532000', '01533400', '01534000', '01534300', '01534500',
       '01536000', '01536500', '01538000', '01539000', '01540500',
       '01541000', '01541200', '01541303', '01541500', '015428

In [24]:
intersect_gdf.to_file(os.path.join("results", "susquehanna_gages_intersect.shp"))

CAMELS流域与Susquehanna的相交，从这里下载CAMELS数据：https://ral.ucar.edu/solutions/products/camels

In [25]:
shpfile_camels = os.path.join("data", "HCDN_nhru_final_671.shp")
camels_gdf = gpd.read_file(shpfile_camels)
camels_gdf

,hru_id,ann_P,lon_cen,lat_cen,AREA,elev_mean,ave_T,july_T,Perimeter,geometry
0,1013500,0.0,-68.56551,47.01169,2.303988e+09,277.493490,None,None,647993,"MULTIPOLYGON (((-68.35650 46.90311, -68.35612 ..."
1,1022500,0.0,-68.07313,44.79691,6.203873e+08,103.604200,None,None,312624,"POLYGON ((-67.97836 44.61310, -67.97800 44.613..."
2,1030500,0.0,-68.14985,45.80527,3.676155e+09,174.433896,None,None,662248,"MULTIPOLYGON (((-67.83991 45.36614, -67.83955 ..."
3,1031500,0.0,-69.58119,45.23568,7.665447e+08,304.743490,None,None,309614,"MULTIPOLYGON (((-69.33810 45.12317, -69.33800 ..."
4,1047000,0.0,-70.16213,44.98744,9.049562e+08,379.779978,None,None,310157,"POLYGON ((-70.10847 45.21669, -70.10858 45.216..."
...,...,...,...,...,...,...,...,...,...,...
666,14309500,0.0,-123.74216,42.83017,2.263143e+08,737.623894,None,None,117484,"POLYGON ((-123.81322 42.89103, -123.81312 42.8..."
667,14316700,0.0,-122.64677,43.45290,5.880250e+08,947.553082,None,None,176861,"POLYGON ((-122.49936 43.47688, -122.49972 43.4..."
668,14325000,0.0,-124.02763,42.79249,4.449257e+08,649.241611,None,None,166110,"POLYGON ((-124.07751 42.89822, -124.07716 42.8..."
669,14362250,0.0,-123.12535,42.16583,4.387790e+07,939.577778,None,None,44972,"POLYGON ((-123.15128 42.19624, -123.15118 42.1..."


In [26]:
camels_gdf.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands

In [27]:
susquehanna4camels = susquehanna_union.to_crs(camels_gdf.crs)

In [28]:
susquehanna4camels.crs == camels_gdf.crs

True

In [29]:
camels_susquehanna_intersects_indices = [i for i in range(camels_gdf.shape[0]) if camels_gdf.geometry[i].intersects(susquehanna4camels.geometry[0])]

In [30]:
camels_susquehanna_intersects_indices

[29,
 35,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 75,
 81,
 282,
 283,
 284]

In [31]:
camels_susquehanna_intersect_gdf = camels_gdf.iloc[camels_susquehanna_intersects_indices]
camels_susquehanna_intersect_gdf

,hru_id,ann_P,lon_cen,lat_cen,AREA,elev_mean,ave_T,july_T,Perimeter,geometry
29,1350140,0.0,-74.53833,42.44596,4.757052e+07,570.437500,None,None,58096,"POLYGON ((-74.58038 42.47954, -74.58047 42.479..."
35,1423000,0.0,-74.87784,42.28871,8.617984e+08,593.927336,None,None,279886,"POLYGON ((-75.08981 42.27290, -75.08988 42.272..."
48,1510000,0.0,-75.80821,42.68791,3.829316e+08,484.344737,None,None,192937,"MULTIPOLYGON (((-75.67783 42.71604, -75.67791 ..."
49,1516500,0.0,-76.96981,41.76926,3.155951e+07,530.437500,None,None,43858,"MULTIPOLYGON (((-76.95200 41.78726, -76.95207 ..."
50,1518862,0.0,-77.62060,41.93231,2.344894e+08,606.673640,None,None,143116,"POLYGON ((-77.52194 41.92576, -77.52196 41.925..."
51,1532000,0.0,-76.69150,41.66388,5.566951e+08,486.784314,None,None,202880,"MULTIPOLYGON (((-76.66934 41.74567, -76.66927 ..."
52,1539000,0.0,-76.34160,41.23694,7.052320e+08,404.009887,None,None,249721,"POLYGON ((-76.12649 41.35491, -76.12656 41.354..."
53,1542810,0.0,-78.28810,41.61150,1.357833e+07,555.461538,None,None,30064,"POLYGON ((-78.29017 41.63813, -78.29023 41.637..."
54,1543000,0.0,-78.29729,41.54218,7.044858e+08,547.577875,None,None,256465,"POLYGON ((-78.33968 41.61911, -78.33974 41.618..."
55,1543500,0.0,-78.35836,41.40397,1.778997e+09,533.470860,None,None,406177,"MULTIPOLYGON (((-78.09733 41.31643, -78.09727 ..."


In [32]:
camels_susquehanna_intersect_gdf.to_file(os.path.join("results", "susquehanna_camels_intersect.shp"))

用这个camels相交的就行。